In [1]:
def modelMatureCell(model,proteinDegrdationRate=1):   
    
    #constrain light-reactions
    model.reactions.get_by_id("Photon_tx").lower_bound = 0
    model.reactions.get_by_id("Photon_tx").upper_bound = 0
    
    #constraint sucrose and ammonium uptake
    model.reactions.get_by_id("Sucrose_tx").lower_bound = 0
    model.reactions.get_by_id("Sucrose_tx").upper_bound = 0
    model.reactions.get_by_id("NH4_tx").lower_bound = 0
    model.reactions.get_by_id("NH4_tx").upper_bound = 0
    
    #set protein turnover rate
    proteinMW = model.metabolites.get_by_id("Protein_b").formula_weight
    model.reactions.get_by_id("Protein_degradation").lower_bound = proteinDegrdationRate/proteinMW
    model.reactions.get_by_id("Protein_degradation").upper_bound = proteinDegrdationRate/proteinMW
    
    #set minimization of glucose uptake as the objective of the  cell
    model.reactions.get_by_id("GLC_tx").objective_coefficient = -1
    
    #run parsimonious FBA to predict flux distribution with minimal metabolic flux
    sol = flux_analysis.parsimonious.pfba(model)
    model.solution = sol
    return model



In [2]:
from cobra import io
from cobra import flux_analysis
from cobra.core import Reaction, Metabolite

#import model from SBML file
model = io.sbml.read_sbml_model("core_model.xml")

#### Add reaction to represent synthesis of protein 
    Transform reaction "Biomass_tx" to "Protein_biomas" reaction by remove inorganic ions and creating a Protein_b metabolite

In [3]:
rxn = model.reactions.get_by_id("Biomass_tx")
model.metabolites.X_Biomass_t.remove_from_model()
rxn.id = "Protein_biomass"
for met in ["Ca_b","K_b","Mg_b"]:
    met = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})

#create metabolite Protein[b]
met = Metabolite("Protein_b",name="Protein[b]")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1

rxn.add_metabolites({met:1})
print(rxn.reaction)

0.456109633051 pALA_b + 0.248708916305 pARG_b + 0.244231166515 pASN_b + 0.244231166515 pASP_b + 0.294390076845 pGLN_b + 0.294390076845 pGLU_b + 0.206565786551 pGLY_b + 0.0787758328997 pHIS_b + 0.164930826186 pILE_b + 0.359404209428 pLEU_b + 0.25644187977 pLYS_b + 0.108683911354 pMET_b + 0.237379330463 pPHE_b + 0.364526466737 pSER_b + 0.206565786551 pTHR_b + 0.165404065972 pTYR_b + 0.312041920884 pVAL_b --> Protein_b


#### Add reaction to represent breakdown of protein to individual amino acids

In [4]:
rxn = Reaction("Protein_degradation")
rxn.add_metabolites({model.metabolites.Protein_b:-1,
                     model.metabolites.L_ALPHA_ALANINE_c:0.456109633051,
                     model.metabolites.ARG_c:0.248708916305,
                     model.metabolites.ASN_c:0.244231166515,
                     model.metabolites.L_ASPARTATE_c:0.244231166515,
                     model.metabolites.GLN_c:0.294390076845,
                     model.metabolites.GLT_c:0.294390076845,
                     model.metabolites.GLY_c:0.206565786551,
                     model.metabolites.HIS_c:0.0787758328997,
                     model.metabolites.ILE_c:0.164930826186,
                     model.metabolites.LEU_c:0.359404209428,
                     model.metabolites.LYS_c:0.25644187977,
                     model.metabolites.MET_c:0.108683911354,
                     model.metabolites.PHE_c:0.237379330463,
                     model.metabolites.SER_c:0.364526466737,
                     model.metabolites.THR_c:0.206565786551,
                     model.metabolites.TYR_c:0.165404065972,
                     model.metabolites.VAL_c:0.312041920884})
rxn.lower_bound = 0
rxn.upper_bound = 1000
model.add_reaction(rxn)
print(rxn.reaction)

Protein_b --> 0.248708916305 ARG_c + 0.244231166515 ASN_c + 0.294390076845 GLN_c + 0.294390076845 GLT_c + 0.206565786551 GLY_c + 0.0787758328997 HIS_c + 0.164930826186 ILE_c + 0.359404209428 LEU_c + 0.25644187977 LYS_c + 0.456109633051 L_ALPHA_ALANINE_c + 0.244231166515 L_ASPARTATE_c + 0.108683911354 MET_c + 0.237379330463 PHE_c + 0.364526466737 SER_c + 0.206565786551 THR_c + 0.165404065972 TYR_c + 0.312041920884 VAL_c


#### Model mature cell

If we assume protein turnover rate is 0.5 $hr^{-1}$ and a cell's protien content is 0.057 $ng.cell^{-1}$, protein degradation rate can be assumed to be **0.029 $ng.cell^{-1}.hr^{-1}$**

In [7]:
model2 = modelMatureCell(model,proteinDegrdationRate=32.96)

#### Analyzing metabolic fluxes

In [8]:
model2.summary()

IN FLUXES                OUT FLUXES              OBJECTIVES
-----------------------  ----------------------  ---------------
OXYGEN_MOLEC...  0.269   CARBON_DIOXI...  0.269  GLC_tx  -0.0448
GLC_e            0.0448  WATER_e          0.269


From the summary, we can see that cell needs to take up atleast 0.0448 $nmol glucose.hr^{-1}$ and has a respiration rate of 0.269 $nmol CO_{2}. hr^{-1}$ under optimal flux distribution

In [9]:
rxn = model2.reactions.Mitochondrial_ATP_Synthase_m
print("mitochondrial ATP synthesis flux = "+str(round(rxn.x,2)*3)+" nmolATP/hr")

rxn1 = model2.reactions.GLU6PDEHYDROG_RXN_p
rxn2 = model2.reactions.GLU6PDEHYDROG_RXN_c
print("pentose phosphate pathway flux = "+str(round(rxn1.x+rxn2.x,5))+" nmol G6P/hr")

rxn3 = model2.reactions.PYRUVDEH_RXN_m
print("mitochondrial pyruvate dehydrogenase = "+str(round(rxn3.x,5))+" nmol pyruvate/hr")

rxn4 = model2.reactions.PYRUVDEH_RXN_p
print("plastidial pyruvate dehydrogenase = "+str(round(rxn4.x,5))+" nmol pyruvate/hr")

mitochondrial ATP synthesis flux = 1.5 nmolATP/hr
pentose phosphate pathway flux = 0.0 nmol G6P/hr
mitochondrial pyruvate dehydrogenase = 0.08959 nmol pyruvate/hr
plastidial pyruvate dehydrogenase = 0.0 nmol pyruvate/hr
